# script that checks some steps of load growth calculation.
so far it only calculates similar fields and you can check if they match visually

In [ ]:
fname = "audit_lg.out"

In [1]:
import pandas as pd
import datetime
from IPython.display import display

In [2]:
pd.options.display.max_columns = None

In [4]:
def decode_out_str_date(s):
    return datetime.datetime.strptime(s, "%Y-%m-%d").date()

def convert_date_fields( dataFrame, srcFields, targetFields=None):
    if (targetFields is None):
        targetFields = srcFields
    for src, target in zip(srcFields, targetFields):        
        tmpcol = '__tmpField_' + src
        dataFrame[tmpcol] = dataFrame[src].apply(decode_out_str_date)
        dataFrame = dataFrame.drop(columns=[src])
        dataFrame = dataFrame.rename(columns = { tmpcol : target}) 
    return dataFrame

In [5]:
auditData = pd.read_csv(fname)

In [6]:
for field in ['day_diff_1', 'day_diff_2', 'day_diff_3', 'day_diff_4', 'day_diff_5']:
    auditData[field].fillna(0, inplace=True)
for field in ['last_gl_date_1', 'last_gl_date_2', 'last_gl_date_3', 'last_gl_date_4', 'last_gl_date_5']:
    auditData[field].fillna(0, inplace=True)    
for field in ['next_gl_date_1', 'next_gl_date_2', 'next_gl_date_3', 'next_gl_date_4', 'next_gl_date_5']:
    auditData[field].fillna(0, inplace=True)
    
for field in ['lg_used_1', 'lg_used_2', 'lg_used_3', 'lg_used_4', 'lg_used_5']:
    auditData[field].fillna(0, inplace=True)    
for field in ['g_factor_1', 'g_factor_2', 'g_factor_3', 'g_factor_4', 'g_factor_5']:
    auditData[field].fillna(1, inplace=True)  

In [7]:
auditData = convert_date_fields(auditData, ['tardate', 'hdate'])
auditData = convert_date_fields(auditData, ['last_gl_date_1', 'next_gl_date_1'])
auditData = convert_date_fields(auditData, ['last_gl_date_2', 'next_gl_date_2'])
auditData = convert_date_fields(auditData, ['last_gl_date_3', 'next_gl_date_3'])
auditData = convert_date_fields(auditData, ['last_gl_date_4', 'next_gl_date_4'])

In [8]:
auditData.columns

Index(['hubcode', 'rank', 'Orgcode', 'ddda', 'temp_diff_abs', 'doys', 'Month',
       'bd', 'dow', 'tarhour', 'tardoy', 'hdow', 'histdoy', 'hhour', 'hload',
       'fct', 'th', 'temp_diff', 'fcdd', 'ddh', 'ddd', 'loadgrowth',
       'growdays', 'growfactor', 'growloads', 'hloads', 'dda', 'ddload',
       'Dloads', 'Tloads', 'Dload', 'lg_used_1', 'day_diff_1', 'g_factor_1',
       'result_load_1', 'lg_used_2', 'day_diff_2', 'g_factor_2',
       'result_load_2', 'last_gl_date_3', 'next_gl_date_3', 'lg_used_3',
       'day_diff_3', 'g_factor_3', 'result_load_3', 'lg_used_4', 'day_diff_4',
       'g_factor_4', 'result_load_4', 'last_gl_date_5', 'next_gl_date_5',
       'lg_used_5', 'day_diff_5', 'g_factor_5', 'result_load_5', 'tardate',
       'hdate', 'last_gl_date_1', 'next_gl_date_1', 'last_gl_date_2',
       'next_gl_date_2', 'last_gl_date_4', 'next_gl_date_4'],
      dtype='object')

In [19]:
auditData['CHK_growdays'] = auditData['tardate'] - auditData['hdate']
auditData['CHK_growdays_1'] = auditData['next_gl_date_1'] - auditData['last_gl_date_1']
auditData['CHK_growdays_2'] = auditData['next_gl_date_2'] - auditData['last_gl_date_2']
auditData['CHK_growdays_3'] = auditData['next_gl_date_3'] - auditData['last_gl_date_3']
auditData['CHK_growdays_4'] = auditData['next_gl_date_4'] - auditData['last_gl_date_4']
auditData['CHK_growdays_as_sum'] = (auditData['day_diff_1'] + auditData['day_diff_2'] + 
                                    auditData['day_diff_3'] + auditData['day_diff_4'] + 
                                    auditData['day_diff_5'])
auditData['CHK_growfactor'] = (auditData['g_factor_1'] * auditData['g_factor_2'] * 
                                    auditData['g_factor_3'] * auditData['g_factor_4'] * 
                                    auditData['g_factor_5'])
auditData['CHK_hloads'] = auditData['CHK_growfactor'] * auditData['hload']

In [10]:
auditData[['CHK_growdays', 'growdays', 'tardate', 'hdate', 'CHK_growdays_as_sum', 
           'last_gl_date_1', 'next_gl_date_1', 
           'last_gl_date_2', 'next_gl_date_2',
           'last_gl_date_4', 'next_gl_date_4',
           'day_diff_1', 'CHK_growdays_1', 'day_diff_4', 'CHK_growdays_4']]

,CHK_growdays,growdays,tardate,hdate,CHK_growdays_as_sum,last_gl_date_1,next_gl_date_1,last_gl_date_2,next_gl_date_2,last_gl_date_4,next_gl_date_4,day_diff_1,CHK_growdays_1,day_diff_4,CHK_growdays_4
0,7305 days,7305,2037-12-08,2017-12-08,7305.0,2017-12-08,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2214,2214 days,708,708 days
1,7306 days,7305,2037-12-08,2017-12-07,7306.0,2017-12-07,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2215,2215 days,708,708 days
2,7305 days,7305,2037-12-08,2017-12-08,7305.0,2017-12-08,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2214,2214 days,708,708 days
3,7662 days,7662,2037-12-08,2016-12-16,7662.0,2016-12-16,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2571,2571 days,708,708 days
4,7649 days,7648,2037-12-08,2016-12-29,7649.0,2016-12-29,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2558,2558 days,708,708 days
5,7305 days,7305,2037-12-08,2017-12-08,7305.0,2017-12-08,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2214,2214 days,708,708 days
6,7662 days,7662,2037-12-08,2016-12-16,7662.0,2016-12-16,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2571,2571 days,708,708 days
7,7305 days,7305,2037-12-08,2017-12-08,7305.0,2017-12-08,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2214,2214 days,708,708 days
8,7648 days,7648,2037-12-08,2016-12-30,7648.0,2016-12-30,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2557,2557 days,708,708 days
9,7627 days,7627,2037-12-08,2017-01-20,7627.0,2017-01-20,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,2536,2536 days,708,708 days


In [11]:
auditData[['growfactor', 'CHK_growfactor', 'hloads', 'CHK_hloads' ]]
# small differences are becaus of ouput rounding

,growfactor,CHK_growfactor,hloads,CHK_hloads
0,1.056419,1.056419,4778.184680,4778.183213
1,1.056422,1.056422,5077.165257,5077.164934
2,1.056419,1.056419,4663.034972,4663.033540
3,1.057453,1.057452,4480.426644,4480.425183
4,1.057415,1.057414,5098.854897,5098.852472
5,1.056419,1.056419,4663.034972,4663.033540
6,1.057453,1.057452,4480.426644,4480.425183
7,1.056419,1.056419,4778.184680,4778.183213
8,1.057412,1.057412,4749.894956,4749.896270
9,1.057351,1.057351,4508.545733,4508.546569


In [12]:
auditData.columns

Index(['hubcode', 'rank', 'Orgcode', 'ddda', 'temp_diff_abs', 'doys', 'Month',
       'bd', 'dow', 'tarhour', 'tardoy', 'hdow', 'histdoy', 'hhour', 'hload',
       'fct', 'th', 'temp_diff', 'fcdd', 'ddh', 'ddd', 'loadgrowth',
       'growdays', 'growfactor', 'growloads', 'hloads', 'dda', 'ddload',
       'Dloads', 'Tloads', 'Dload', 'lg_used_1', 'day_diff_1', 'g_factor_1',
       'result_load_1', 'lg_used_2', 'day_diff_2', 'g_factor_2',
       'result_load_2', 'last_gl_date_3', 'next_gl_date_3', 'lg_used_3',
       'day_diff_3', 'g_factor_3', 'result_load_3', 'lg_used_4', 'day_diff_4',
       'g_factor_4', 'result_load_4', 'last_gl_date_5', 'next_gl_date_5',
       'lg_used_5', 'day_diff_5', 'g_factor_5', 'result_load_5', 'tardate',
       'hdate', 'last_gl_date_1', 'next_gl_date_1', 'last_gl_date_2',
       'next_gl_date_2', 'last_gl_date_4', 'next_gl_date_4', 'CHK_growdays',
       'CHK_growdays_1', 'CHK_growdays_4', 'CHK_growdays_as_sum',
       'CHK_growfactor', 'CHK_hloads'],
    

In [20]:
auditData.head()

,hubcode,rank,Orgcode,ddda,temp_diff_abs,doys,Month,bd,dow,tarhour,tardoy,hdow,histdoy,hhour,hload,fct,th,temp_diff,fcdd,ddh,ddd,loadgrowth,growdays,growfactor,growloads,hloads,dda,ddload,Dloads,Tloads,Dload,lg_used_1,day_diff_1,g_factor_1,result_load_1,lg_used_2,day_diff_2,g_factor_2,result_load_2,lg_used_3,day_diff_3,g_factor_3,result_load_3,lg_used_4,day_diff_4,g_factor_4,result_load_4,last_gl_date_5,next_gl_date_5,lg_used_5,day_diff_5,g_factor_5,result_load_5,tardate,hdate,last_gl_date_1,next_gl_date_1,last_gl_date_2,next_gl_date_2,last_gl_date_4,next_gl_date_4,CHK_growdays,CHK_growdays_1,CHK_growdays_4,CHK_growdays_as_sum,CHK_growfactor,CHK_hloads,last_gl_date_3,next_gl_date_3,CHK_growdays_2,CHK_growdays_3
0,CO,1,PSCO,0.0,0.0,0,12,1,5,1,342,5,342,1,4523,36.0,36.0,0.0,29.0,29,0.0,0.002812,7305,1.056419,255.184680,4778.184680,12,0.0,4778.184680,172014.6485,138567.3557,0.001,2214,1.006081,4550.50,0.004,2192,1.024264,4660.92,0.0035,2191,1.021194,4759.70,0.002,708,1.003883,4778.18,0.0,0.0,0.0,0.0,1.0,NaN,2037-12-08,2017-12-08,2017-12-08,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,7305 days,2214 days,708 days,7305.0,1.056419,4778.183213,2029-12-31,2035-12-31,2192 days,2191 days
1,CO,2,PSCO,1.0,1.0,0,12,1,5,1,342,4,341,24,4806,36.0,35.0,1.0,29.0,30,-1.0,0.002812,7305,1.056422,271.165257,5077.165257,12,-12.0,5065.165257,182345.9492,146889.7924,0.001,2215,1.006084,4835.24,0.004,2192,1.024264,4952.56,0.0035,2191,1.021194,5057.53,0.002,708,1.003883,5077.17,0.0,0.0,0.0,0.0,1.0,NaN,2037-12-08,2017-12-07,2017-12-07,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,7306 days,2215 days,708 days,7306.0,1.056422,5077.164934,2029-12-31,2035-12-31,2192 days,2191 days
2,CO,3,PSCO,1.0,1.0,0,12,1,5,1,342,5,342,2,4414,36.0,37.0,-1.0,29.0,28,1.0,0.002812,7305,1.056419,249.034972,4663.034972,12,12.0,4675.034972,168301.2590,135576.0142,0.001,2214,1.006081,4440.84,0.004,2192,1.024264,4548.59,0.0035,2191,1.021194,4645.00,0.002,708,1.003883,4663.03,0.0,0.0,0.0,0.0,1.0,NaN,2037-12-08,2017-12-08,2017-12-08,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,7305 days,2214 days,708 days,7305.0,1.056419,4663.033540,2029-12-31,2035-12-31,2192 days,2191 days
3,CO,4,PSCO,1.0,1.0,8,12,1,5,1,342,5,351,2,4237,36.0,37.0,-1.0,29.0,28,1.0,0.002812,7662,1.057453,243.426644,4480.426644,12,12.0,4492.426644,161727.3592,130280.3727,0.001,2571,1.007065,4266.94,0.004,2192,1.024264,4370.47,0.0035,2191,1.021194,4463.10,0.002,708,1.003883,4480.43,0.0,0.0,0.0,0.0,1.0,NaN,2037-12-08,2016-12-16,2016-12-16,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,7662 days,2571 days,708 days,7662.0,1.057452,4480.425183,2029-12-31,2035-12-31,2192 days,2191 days
4,CO,5,PSCO,1.0,1.0,22,12,1,5,1,342,4,364,24,4822,36.0,37.0,-1.0,29.0,28,1.0,0.002812,7648,1.057415,276.854897,5098.854897,12,12.0,5110.854897,183990.7763,148214.7920,0.001,2558,1.007029,4855.90,0.004,2192,1.024264,4973.72,0.0035,2191,1.021194,5079.13,0.002,708,1.003883,5098.85,0.0,0.0,0.0,0.0,1.0,NaN,2037-12-08,2016-12-29,2016-12-29,2023-12-31,2023-12-31,2029-12-31,2035-12-31,2037-12-08,7649 days,2558 days,708 days,7649.0,1.057414,5098.852472,2029-12-31,2035-12-31,2192 days,2191 days


In [14]:
lgBasicFields = ['Orgcode','tardate','hdate','hload','loadgrowth', 'growdays', 'growfactor', 'growloads', 'hloads' ]
lgAdvFields = ['last_gl_date_1', 'next_gl_date_1', 'lg_used_1', 'day_diff_1', 'g_factor_1', 'result_load_1',
              'last_gl_date_2', 'next_gl_date_2', 'lg_used_2', 'day_diff_2', 'g_factor_2', 'result_load_2',
              'last_gl_date_3', 'next_gl_date_3', 'lg_used_3', 'day_diff_3', 'g_factor_3', 'result_load_3',
              'last_gl_date_4', 'next_gl_date_4', 'lg_used_4', 'day_diff_4', 'g_factor_4', 'result_load_4',
              'last_gl_date_5', 'next_gl_date_5', 'lg_used_5', 'day_diff_5', 'g_factor_5', 'result_load_5']
lgBasicData = auditData[lgBasicFields]
lgData = auditData[lgBasicFields + lgAdvFields]

In [15]:
lgData.describe().T

,count,mean,std,min,25%,50%,75%,max
hload,75600.0,2766.096349,2817.134975,55.000000,833.000000,1905.000000,3321.000000,16834.000000
loadgrowth,75600.0,-0.002558,0.024484,-0.050000,-0.020000,0.002075,0.010000,0.050000
growdays,75600.0,7781.373730,242.532107,7305.000000,7647.000000,7690.000000,8026.000000,8096.000000
growfactor,75600.0,1.057782,0.007065,1.044756,1.056521,1.057574,1.059672,1.069659
growloads,75600.0,158.130581,153.869423,2.474640,54.583430,111.840769,204.122993,945.486595
hloads,75600.0,2924.226931,2969.136873,57.474640,888.363217,2013.461875,3509.772240,17591.903325
lg_used_1,75600.0,0.001450,0.000650,0.001000,0.001000,0.001000,0.002000,0.002500
day_diff_1,75600.0,2680.374325,242.139129,2213.000000,2541.000000,2586.000000,2922.000000,3005.000000
g_factor_1,75600.0,1.010736,0.005020,1.006078,1.007040,1.008025,1.016001,1.020769
result_load_1,75600.0,2795.548025,2846.737368,55.340000,847.815000,1925.550000,3354.575000,16938.410000


In [16]:
lgData.head()


,Orgcode,tardate,hdate,hload,loadgrowth,growdays,growfactor,growloads,hloads,last_gl_date_1,next_gl_date_1,lg_used_1,day_diff_1,g_factor_1,result_load_1,last_gl_date_2,next_gl_date_2,lg_used_2,day_diff_2,g_factor_2,result_load_2,last_gl_date_3,next_gl_date_3,lg_used_3,day_diff_3,g_factor_3,result_load_3,last_gl_date_4,next_gl_date_4,lg_used_4,day_diff_4,g_factor_4,result_load_4,last_gl_date_5,next_gl_date_5,lg_used_5,day_diff_5,g_factor_5,result_load_5
0,PSCO,2037-12-08,2017-12-08,4523,0.002812,7305,1.056419,255.184680,4778.184680,2017-12-08,2023-12-31,0.001,2214,1.006081,4550.50,2023-12-31,2029-12-31,0.004,2192,1.024264,4660.92,2029-12-31,2035-12-31,0.0035,2191,1.021194,4759.70,2035-12-31,2037-12-08,0.002,708,1.003883,4778.18,0.0,0.0,0.0,0.0,1.0,NaN
1,PSCO,2037-12-08,2017-12-07,4806,0.002812,7305,1.056422,271.165257,5077.165257,2017-12-07,2023-12-31,0.001,2215,1.006084,4835.24,2023-12-31,2029-12-31,0.004,2192,1.024264,4952.56,2029-12-31,2035-12-31,0.0035,2191,1.021194,5057.53,2035-12-31,2037-12-08,0.002,708,1.003883,5077.17,0.0,0.0,0.0,0.0,1.0,NaN
2,PSCO,2037-12-08,2017-12-08,4414,0.002812,7305,1.056419,249.034972,4663.034972,2017-12-08,2023-12-31,0.001,2214,1.006081,4440.84,2023-12-31,2029-12-31,0.004,2192,1.024264,4548.59,2029-12-31,2035-12-31,0.0035,2191,1.021194,4645.00,2035-12-31,2037-12-08,0.002,708,1.003883,4663.03,0.0,0.0,0.0,0.0,1.0,NaN
3,PSCO,2037-12-08,2016-12-16,4237,0.002812,7662,1.057453,243.426644,4480.426644,2016-12-16,2023-12-31,0.001,2571,1.007065,4266.94,2023-12-31,2029-12-31,0.004,2192,1.024264,4370.47,2029-12-31,2035-12-31,0.0035,2191,1.021194,4463.10,2035-12-31,2037-12-08,0.002,708,1.003883,4480.43,0.0,0.0,0.0,0.0,1.0,NaN
4,PSCO,2037-12-08,2016-12-29,4822,0.002812,7648,1.057415,276.854897,5098.854897,2016-12-29,2023-12-31,0.001,2558,1.007029,4855.90,2023-12-31,2029-12-31,0.004,2192,1.024264,4973.72,2029-12-31,2035-12-31,0.0035,2191,1.021194,5079.13,2035-12-31,2037-12-08,0.002,708,1.003883,5098.85,0.0,0.0,0.0,0.0,1.0,NaN


In [25]:
def calc_growload1(row):
    return pow(1.+row['lg_used_1'],  row['day_diff_1']/365)

def calc_growload2(row):
    return pow(1.+row['lg_used_2'],  row['day_diff_2']/365)

def calc_growload4(row):
    return pow(1.+row['lg_used_4'],  row['day_diff_4']/365)


lgData['CHK_g_factor_1'] = lgData.apply(calc_growload1, axis=1)
lgData['CHK_g_factor_2'] = lgData.apply(calc_growload2, axis=1)
lgData['CHK_g_factor_4'] = lgData.apply(calc_growload4, axis=1)

In [26]:
lgData[['g_factor_1', 'CHK_g_factor_1','g_factor_2', 'CHK_g_factor_2','g_factor_4', 'CHK_g_factor_4']]

,g_factor_1,CHK_g_factor_1,g_factor_2,CHK_g_factor_2,g_factor_4,CHK_g_factor_4
0,1.006081,1.006081,1.024264,1.024264,1.003883,1.003883
1,1.006084,1.006084,1.024264,1.024264,1.003883,1.003883
2,1.006081,1.006081,1.024264,1.024264,1.003883,1.003883
3,1.007065,1.007065,1.024264,1.024264,1.003883,1.003883
4,1.007029,1.007029,1.024264,1.024264,1.003883,1.003883
5,1.006081,1.006081,1.024264,1.024264,1.003883,1.003883
6,1.007065,1.007065,1.024264,1.024264,1.003883,1.003883
7,1.006081,1.006081,1.024264,1.024264,1.003883,1.003883
8,1.007027,1.007027,1.024264,1.024264,1.003883,1.003883
9,1.006969,1.006969,1.024264,1.024264,1.003883,1.003883
